# Cleaning SPOTT pre-processed file

This notebook reads the pre-processed SPOTT dataset from local data folder and performs the cleaning of country, IDs and company's names. The final result is saved locally and in the SPOTT S3 bucket.

In [19]:
import pandas as pd
import os

from financial_entity_cleaner.location import CountryCleaner
from financial_entity_cleaner.company import CompanyNameCleaner
from financial_entity_cleaner.id import BankingIdCleaner
pd.set_option('display.max_columns',50)
from dotenv import dotenv_values, load_dotenv

In [20]:
env_var = dotenv_values('../../.env')

In [21]:
import boto3
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=env_var['S3_ENDPOINT'],
    aws_access_key_id=env_var['S3_ACCESS_KEY'],
    aws_secret_access_key=env_var['S3_SECRET_KEY'],
)
bucket_name = env_var['S3_BUCKET']
bucket = s3_resource.Bucket(bucket_name)

## 1. Read SPOTT pre-processed file

In [22]:
# Save locally to "data" folder
Spott = bucket.Object('SPOTT/pre_processed/spott_pre_processed.csv').get()['Body']
spott_df = pd.read_csv(Spott, encoding='utf-8', delimiter=',', low_memory=False)

In [23]:
spott_df.head()

,company_name,parent_company,subsidiaries,market_cap,bbg_ticker,reuters_ticker,isin,sedol,landbank_ha,gpsnr_member,activities,small_holders,locations,headquarter_country,notes,website,landbank_oil_ha,rspo_member,member_since
0,Austindo Nusantara Jaya Tbk PT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bakrie Sumatera Plantations Tbk PT,PT Bakrie & Brothers Tbk,PT Huma Indah Mekar (HIM),2.000000e+07,UNSP IJ Equity,UNSP.JK,ID1000099708,NaN,19789.0,No,Natural rubber cultivation and processing,Unclear whether company has industrial plantat...,Indonesia (Sumatra),Indonesia,NaN,https://www.bakriesumatera.com/,NaN,NaN,NaN
2,Bridgestone Corporation,NaN,"Firestone Natural Rubber Company, LLC",2.919700e+10,5108 JP Equity,5108.T,JP3830800003,NaN,NaN,Yes,"Natural rubber cultivation, processing and dis...",Company has industrial plantation suppliers; C...,"Liberia (Harbel), Indonesia (Kalimantan, Sumat...",Japan,NaN,https://www.bridgestone.com,NaN,NaN,NaN
3,Cheng Shin Rubber Industry Co Ltd (正新橡胶工业股份有限公司),NaN,"Cheng Shin Rubber (China) Co., Ltd. and Cheng ...",4.124000e+09,2105 TT Equity,NaN,TW0002105004,NaN,NaN,No,"Natural rubber manufacturing, trading and dist...",Company has scheme/outgrower smallholder suppl...,"China (Xiamen, Chongqing, Zhangzhou)",Taiwan,NaN,https://www.csttires.com/int/about-cst/,NaN,NaN,NaN
4,Continental AG,NaN,"General Tire, Continental Reifen Deutschland GmbH",1.580400e+10,CON GR Equity,NaN,DE0005439004,NaN,NaN,No,Natural rubber manufacturing,Company has scheme/outgrower smallholder suppl...,"Germany (Hanover, Korbatch), Czech Republic (O...",Germany,NaN,https://www.continental.com/en/,NaN,NaN,NaN


## 2. Cleaning County information

In [24]:
# Create an object based on CountryCleaner() class
country_cleaner_obj=CountryCleaner()

In [25]:
country_cleaner_obj.letter_case='lower'
country_cleaner_obj.output_info = [CountryCleaner.ATTRIBUTE_ALPHA2,
                                   CountryCleaner.ATTRIBUTE_SHORT_NAME]

In [26]:
spott_df = country_cleaner_obj.clean_df(df=spott_df, cols=['headquarter_country'], output_names_as= 'suffix')

Normalizing countries...100%|██████████████████████████████████████████████████| 245/245 [00:00<00:00, 5008.80it/s]


In [27]:
spott_df.drop('headquarter_country', axis=1, inplace=True)

In [28]:
spott_df.head()

,company_name,parent_company,subsidiaries,market_cap,bbg_ticker,reuters_ticker,isin,sedol,landbank_ha,gpsnr_member,activities,small_holders,locations,notes,website,landbank_oil_ha,rspo_member,member_since,headquarter_country_short_name,headquarter_country_alpha2
0,Austindo Nusantara Jaya Tbk PT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bakrie Sumatera Plantations Tbk PT,PT Bakrie & Brothers Tbk,PT Huma Indah Mekar (HIM),2.000000e+07,UNSP IJ Equity,UNSP.JK,ID1000099708,NaN,19789.0,No,Natural rubber cultivation and processing,Unclear whether company has industrial plantat...,Indonesia (Sumatra),NaN,https://www.bakriesumatera.com/,NaN,NaN,NaN,indonesia,id
2,Bridgestone Corporation,NaN,"Firestone Natural Rubber Company, LLC",2.919700e+10,5108 JP Equity,5108.T,JP3830800003,NaN,NaN,Yes,"Natural rubber cultivation, processing and dis...",Company has industrial plantation suppliers; C...,"Liberia (Harbel), Indonesia (Kalimantan, Sumat...",NaN,https://www.bridgestone.com,NaN,NaN,NaN,japan,jp
3,Cheng Shin Rubber Industry Co Ltd (正新橡胶工业股份有限公司),NaN,"Cheng Shin Rubber (China) Co., Ltd. and Cheng ...",4.124000e+09,2105 TT Equity,NaN,TW0002105004,NaN,NaN,No,"Natural rubber manufacturing, trading and dist...",Company has scheme/outgrower smallholder suppl...,"China (Xiamen, Chongqing, Zhangzhou)",NaN,https://www.csttires.com/int/about-cst/,NaN,NaN,NaN,taiwan,tw
4,Continental AG,NaN,"General Tire, Continental Reifen Deutschland GmbH",1.580400e+10,CON GR Equity,NaN,DE0005439004,NaN,NaN,No,Natural rubber manufacturing,Company has scheme/outgrower smallholder suppl...,"Germany (Hanover, Korbatch), Czech Republic (O...",NaN,https://www.continental.com/en/,NaN,NaN,NaN,germany,de


## 3. Cleaning IDs

In [29]:
# Create an object based on the BankingIdCleaner() class
id_cleaner_obj = BankingIdCleaner()

In [30]:
# Setup cleaning properties
id_cleaner_obj.invalid_ids_as_nan = True
id_cleaner_obj.validation_as_categorical = True
id_cleaner_obj.output_lettercase = "BankingIdCleaner.UPPER_LETTER_CASE"

In [31]:
spott_df = id_cleaner_obj.clean_df(spott_df, cols=['isin', 'sedol'], remove_cols= True, 
                                   output_names_as= 'suffix', types = ['isin', 'sedol'])

Normalizing IDs...100%|██████████████████████████████████████████████████| 245/245 [00:00<00:00, 3116.18it/s]


In [32]:
spott_df.head()

,company_name,parent_company,subsidiaries,market_cap,bbg_ticker,reuters_ticker,landbank_ha,gpsnr_member,activities,small_holders,locations,notes,website,landbank_oil_ha,rspo_member,member_since,headquarter_country_short_name,headquarter_country_alpha2,isin_cleaned_id,isin_isvalid_id,sedol_cleaned_id,sedol_isvalid_id
0,Austindo Nusantara Jaya Tbk PT,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bakrie Sumatera Plantations Tbk PT,PT Bakrie & Brothers Tbk,PT Huma Indah Mekar (HIM),2.000000e+07,UNSP IJ Equity,UNSP.JK,19789.0,No,Natural rubber cultivation and processing,Unclear whether company has industrial plantat...,Indonesia (Sumatra),NaN,https://www.bakriesumatera.com/,NaN,NaN,NaN,indonesia,id,ID1000099708,1.0,NaN,NaN
2,Bridgestone Corporation,NaN,"Firestone Natural Rubber Company, LLC",2.919700e+10,5108 JP Equity,5108.T,NaN,Yes,"Natural rubber cultivation, processing and dis...",Company has industrial plantation suppliers; C...,"Liberia (Harbel), Indonesia (Kalimantan, Sumat...",NaN,https://www.bridgestone.com,NaN,NaN,NaN,japan,jp,JP3830800003,1.0,NaN,NaN
3,Cheng Shin Rubber Industry Co Ltd (正新橡胶工业股份有限公司),NaN,"Cheng Shin Rubber (China) Co., Ltd. and Cheng ...",4.124000e+09,2105 TT Equity,NaN,NaN,No,"Natural rubber manufacturing, trading and dist...",Company has scheme/outgrower smallholder suppl...,"China (Xiamen, Chongqing, Zhangzhou)",NaN,https://www.csttires.com/int/about-cst/,NaN,NaN,NaN,taiwan,tw,TW0002105004,1.0,NaN,NaN
4,Continental AG,NaN,"General Tire, Continental Reifen Deutschland GmbH",1.580400e+10,CON GR Equity,NaN,NaN,No,Natural rubber manufacturing,Company has scheme/outgrower smallholder suppl...,"Germany (Hanover, Korbatch), Czech Republic (O...",NaN,https://www.continental.com/en/,NaN,NaN,NaN,germany,de,DE0005439004,1.0,NaN,NaN


## 4. Getting LEI from cleaned ISIN

In [33]:
# Get the ISIN - LEI dataset from GLEIF S3
isin_lei = bucket.Object('GLEIF/raw/ISIN_LEI_20221101.csv').get()['Body']
isin_lei_df = pd.read_csv(isin_lei, encoding='utf-8', delimiter=',', low_memory=False)

In [34]:
isin_lei_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728459 entries, 0 to 7728458
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   LEI     object
 1   ISIN    object
dtypes: object(2)
memory usage: 117.9+ MB


In [35]:
isin_lei_df.columns = ['lei', 'isin']

In [36]:
isin_lei_df.head()

,lei,isin
0,724500V6UOK62XEZ2L78,NLEN03247298
1,529900W18LQJJN6SJ336,DE000SN4ESX0
2,724500V6UOK62XEZ2L78,NLENX7751979
3,724500V6UOK62XEZ2L78,NLICE3689953
4,959800T2W59YXMVKRU25,ES0A04191961


In [37]:
# Merge ISIN to get LEI
spott_df = spott_df.merge(isin_lei_df, left_on=['isin_cleaned_id'], right_on=['isin'], how='left')

In [38]:
spott_df.drop('isin', axis=1, inplace=True)

In [39]:
spott_df = id_cleaner_obj.clean_df(spott_df, cols=['lei'], remove_cols= True, 
                                   output_names_as= 'suffix', types = ['lei'])

Normalizing IDs...100%|██████████████████████████████████████████████████| 245/245 [00:00<00:00, 5496.21it/s]


In [40]:
spott_df.head()

,company_name,parent_company,subsidiaries,market_cap,bbg_ticker,reuters_ticker,landbank_ha,gpsnr_member,activities,small_holders,locations,notes,website,landbank_oil_ha,rspo_member,member_since,headquarter_country_short_name,headquarter_country_alpha2,isin_cleaned_id,isin_isvalid_id,sedol_cleaned_id,sedol_isvalid_id,lei_cleaned_id,lei_isvalid_id
0,Austindo Nusantara Jaya Tbk PT,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bakrie Sumatera Plantations Tbk PT,PT Bakrie & Brothers Tbk,PT Huma Indah Mekar (HIM),2.000000e+07,UNSP IJ Equity,UNSP.JK,19789.0,No,Natural rubber cultivation and processing,Unclear whether company has industrial plantat...,Indonesia (Sumatra),NaN,https://www.bakriesumatera.com/,NaN,NaN,NaN,indonesia,id,ID1000099708,1.0,NaN,NaN,NaN,NaN
2,Bridgestone Corporation,NaN,"Firestone Natural Rubber Company, LLC",2.919700e+10,5108 JP Equity,5108.T,NaN,Yes,"Natural rubber cultivation, processing and dis...",Company has industrial plantation suppliers; C...,"Liberia (Harbel), Indonesia (Kalimantan, Sumat...",NaN,https://www.bridgestone.com,NaN,NaN,NaN,japan,jp,JP3830800003,1.0,NaN,NaN,549300DHPOF90OYYD780,1.0
3,Cheng Shin Rubber Industry Co Ltd (正新橡胶工业股份有限公司),NaN,"Cheng Shin Rubber (China) Co., Ltd. and Cheng ...",4.124000e+09,2105 TT Equity,NaN,NaN,No,"Natural rubber manufacturing, trading and dist...",Company has scheme/outgrower smallholder suppl...,"China (Xiamen, Chongqing, Zhangzhou)",NaN,https://www.csttires.com/int/about-cst/,NaN,NaN,NaN,taiwan,tw,TW0002105004,1.0,NaN,NaN,NaN,NaN
4,Continental AG,NaN,"General Tire, Continental Reifen Deutschland GmbH",1.580400e+10,CON GR Equity,NaN,NaN,No,Natural rubber manufacturing,Company has scheme/outgrower smallholder suppl...,"Germany (Hanover, Korbatch), Czech Republic (O...",NaN,https://www.continental.com/en/,NaN,NaN,NaN,germany,de,DE0005439004,1.0,NaN,NaN,529900A7YD9C0LLXM621,1.0


## 5. Cleaning Company's names

In [41]:
# Create a CompanyNameCleaner object
company_cleaner_obj = CompanyNameCleaner()

In [42]:
# Set cleaner properties
company_cleaner_obj.normalize_legal_terms = True
company_cleaner_obj.letter_case="lower"
company_cleaner_obj.remove_accents = True
company_cleaner_obj.default_cleaning_rules = ['place_word_the_at_the_beginning',
                                                'remove_words_in_asterisk', 
                                                'remove_words_in_parentheses',
                                                'remove_question_marks_in_parentheses', 
                                                'replace_hyphen_by_space', 
                                                'replace_underscore_by_space', 
                                                'remove_text_puctuation_except_dot', 
                                                'remove_math_symbols', 
                                                'remove_parentheses', 
                                                'remove_brackets', 
                                                'remove_curly_brackets', 
                                                'remove_single_quote_next_character', 
                                                'remove_double_quote', 
                                                'enforce_single_space_between_words']
company_cleaner_obj.post_cleaning_rules = ['remove_all_punctuation']

In [43]:
spott_df = company_cleaner_obj.clean_df(spott_df, 'company_name', 'company_name_clean', 'headquarter_country_alpha2', 'True')

Cleaning company name...100%|██████████████████████████████████████████████████| 38/38 [00:00<00:00, 212.51it/s]


In [44]:
spott_df.reset_index(inplace=True)
spott_df = spott_df.rename(columns = {'index':'company_id'})

In [45]:
spott_df.head()

,company_id,company_name,parent_company,subsidiaries,market_cap,bbg_ticker,reuters_ticker,landbank_ha,gpsnr_member,activities,small_holders,locations,notes,website,landbank_oil_ha,rspo_member,member_since,headquarter_country_short_name,headquarter_country_alpha2,isin_cleaned_id,isin_isvalid_id,sedol_cleaned_id,sedol_isvalid_id,lei_cleaned_id,lei_isvalid_id,company_name_clean
0,0,Austindo Nusantara Jaya Tbk PT,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,austindo nusantara jaya tbk pt
1,1,Bakrie Sumatera Plantations Tbk PT,PT Bakrie & Brothers Tbk,PT Huma Indah Mekar (HIM),2.000000e+07,UNSP IJ Equity,UNSP.JK,19789.0,No,Natural rubber cultivation and processing,Unclear whether company has industrial plantat...,Indonesia (Sumatra),NaN,https://www.bakriesumatera.com/,NaN,NaN,NaN,indonesia,id,ID1000099708,1.0,NaN,NaN,NaN,NaN,bakrie sumatera plantations tbk perseroan terb...
2,2,Bridgestone Corporation,NaN,"Firestone Natural Rubber Company, LLC",2.919700e+10,5108 JP Equity,5108.T,NaN,Yes,"Natural rubber cultivation, processing and dis...",Company has industrial plantation suppliers; C...,"Liberia (Harbel), Indonesia (Kalimantan, Sumat...",NaN,https://www.bridgestone.com,NaN,NaN,NaN,japan,jp,JP3830800003,1.0,NaN,NaN,549300DHPOF90OYYD780,1.0,bridgestone corporation
3,3,Cheng Shin Rubber Industry Co Ltd (正新橡胶工业股份有限公司),NaN,"Cheng Shin Rubber (China) Co., Ltd. and Cheng ...",4.124000e+09,2105 TT Equity,NaN,NaN,No,"Natural rubber manufacturing, trading and dist...",Company has scheme/outgrower smallholder suppl...,"China (Xiamen, Chongqing, Zhangzhou)",NaN,https://www.csttires.com/int/about-cst/,NaN,NaN,NaN,taiwan,tw,TW0002105004,1.0,NaN,NaN,NaN,NaN,cheng shin rubber industry co limited
4,4,Continental AG,NaN,"General Tire, Continental Reifen Deutschland GmbH",1.580400e+10,CON GR Equity,NaN,NaN,No,Natural rubber manufacturing,Company has scheme/outgrower smallholder suppl...,"Germany (Hanover, Korbatch), Czech Republic (O...",NaN,https://www.continental.com/en/,NaN,NaN,NaN,germany,de,DE0005439004,1.0,NaN,NaN,529900A7YD9C0LLXM621,1.0,continental aktiengesellschaft


In [46]:
print('Total companies in SPOTT {}'.format(spott_df.shape[0]))

Total companies in SPOTT 245


## 6. Save cleaned SPOTT

In [47]:
# Save locally to "data" folder
saved_path = "../../../dataset/cleaned/"
filename = "spott_cleaned.csv"
spott_filename = os.path.join(saved_path, filename)
spott_df.to_csv(spott_filename, header=True, index=False)

In [48]:
# Save final results to S3
s3_filename = 'SPOTT/cleaned/spott_cleaned.csv'
s3_resource.meta.client.upload_file(Filename=spott_filename, Bucket=bucket_name, Key=s3_filename)